In [1]:
dataset = 'lastfm'

domain = {
    'lastfm': 'music',
    'dbbook': 'book',
    'movielens': 'movie'
}[dataset]

items = {
    'lastfm': 'artist',
    'dbbook': 'book',
    'movielens': 'movie'
}[dataset]
import os
datapath = os.path.join('..', 'data', dataset)

special_token = '<|reserved_special_token_22|>'

resp_frac = 0.5

# Set names

In [2]:
import pandas as pd
import os
import random
import numpy as np
import swifter  

In [3]:
train_text = pd.read_pickle(os.path.join(datapath, f'{dataset}_train_dataset_all_items.pkl'))

In [5]:
item_desc_df = pd.read_json(os.path.join(datapath, f'{dataset}_domain_kgraph_all_items.jsonl'), lines=True) 
item_desc_df['target_id'] = item_desc_df['target_id'].astype(int)
item_desc_dict_graph = item_desc_df.set_index('target_id').to_dict()['text']

item_desc_df = pd.read_json(os.path.join(datapath, f'{dataset}_domain_kn_all_items.jsonl'), lines=True) 
item_desc_df['target_id'] = item_desc_df['target_id'].astype(int)
item_desc_dict_text = item_desc_df.set_index('target_id').to_dict()['text']

ValueError: Expected object or value

In [73]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../llama3")
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.chat_template =  "{% set ns = namespace(i=0) %}" \
#                            "{% for message in messages %}" \
#                                "{% if message['role'] == 'system' and ns.i == 0 %}" \
#                                       "{{ bos_token +' [INST] <<SYS>>\n' }}" \
#                                       "{{ message['content'] + ' <</SYS>>\n'}}" \
#                                "{% elif message['role'] == 'user' %}" \
#                                    "{{ message['content'] + ' [/INST]\n'}}" \
#                                "<|reserved_special_token_22|>\n"\
#                                "{% elif message['role'] == 'assistant' %}" \
#                                    "{{ message['content'] + ' ' + eos_token }}" \
#                                "{% endif %}" \
#                                "{% set ns.i = ns.i+1 %}" \
#                            "{% endfor %}"
#
tokenizer.chat_template =  "{% set ns = namespace(i=0) %}" \
                            "{% for message in messages %}" \
                                "{% if message['role'] == 'system' and ns.i == 0 %}" \
                                       "{{ bos_token +' [INST] <<SYS>>\n' }}" \
                                       "{{ message['content'] + ' <</SYS>>\n'}}" \
                                "{% elif message['role'] == 'user' %}" \
                                    "{{ message['content'] + ' [/INST]\n'}}" \
                                "{% elif message['role'] == 'assistant' %}" \
                                    "{{ message['content'] + '' + eos_token }}" \
                                "{% endif %}" \
                                "{% set ns.i = ns.i+1 %}" \
                            "{% endfor %}"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [74]:
prompts = train_text['prompt'].tolist() * 3
descriptions = list(item_desc_dict_graph.values()) + list(item_desc_dict_text.values()) 
descriptions = [f'{tokenizer.bos_token} ' + x + f' {tokenizer.eos_token}' for x in descriptions]

In [75]:
import random 

random.seed(22)
train_set = prompts + descriptions

random.shuffle(train_set)
len(train_set)

8560

In [76]:
import json

with open(os.path.join(datapath, f'{dataset}_train_set_graph_text.jsonl'), 'w') as outfile:
    for x in train_set:
        json.dump({'text':x}, outfile)
        outfile.write('\n')